In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Activation
from sklearn.metrics import mean_squared_error, mean_absolute_error

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
doge = pd.read_csv("doge.csv")
doge.head()

In [ ]:
doge.shape

In [ ]:
doge.info()

In [ ]:
total = doge.isnull().sum().sort_values(ascending=False)
percent = (doge.isnull().sum()/doge.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data

In [ ]:
#Plotting correlation
plt.figure(figsize=(7,7))
corr=doge[doge.columns[1:]].corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(doge[doge.columns[1:]].corr(), mask=mask, vmax=.3, center=0,
square=True, linewidths=.5,annot=True)
plt.show()

In [ ]:
fig,axes = plt.subplots(3,2,figsize = (10,10))
fig.suptitle("Count plot for categorical features")
#Open
sns.violinplot(ax=axes[0,0],data=doge,x='Open',color='#4e89ae')
#High
sns.violinplot(ax=axes[0,1],data=doge,x='High',color='#c56183')
#Low
sns.violinplot(ax=axes[1,0],data=doge,x='Low',color='#ff8000')
#Close
sns.violinplot(ax=axes[1,1],data=doge,x='Close',color='#ffd700')
#Adj Close
sns.violinplot(ax=axes[2,0],data=doge,x='Adj Close',color='#7cfC00')
#Volume
sns.violinplot(ax=axes[2,1],data=doge,x='Volume',color='#00FFFF')

In [ ]:
doge['Volume'].fillna(value=0, inplace=True)
doge['Open'].fillna(method='ffill', inplace=True)
doge['High'].fillna(method='ffill', inplace=True)
doge['Low'].fillna(method='ffill', inplace=True)
doge['Close'].fillna(method='ffill', inplace=True)
doge['Adj Close'].fillna(method='ffill', inplace=True)

In [ ]:
doge['Date'] = pd.to_datetime(doge['Date'])
doge.rename(columns = {'Close':'Price'}, inplace = True)

In [ ]:
doge['Date'] = doge['Date'].dt.tz_localize(None)
doge = doge.set_index('Date')
doge = doge[['Price']]

In [ ]:
#Splitting data
split_date = '2018-06-25'
data_train = doge.loc[doge.index <= split_date].copy()
data_test = doge.loc[doge.index > split_date].copy()


In [ ]:
#Data preprocessing
training_set = data_train.values
training_set = np.reshape(training_set, (len(training_set), 1))
sc = MinMaxScaler()
training_set = sc.fit_transform(training_set)
X_train = training_set[0:len(training_set)-1]
y_train = training_set[1:len(training_set)]
X_train = np.reshape(X_train, (len(X_train), 1, 1))

In [ ]:
#Historical price
_ = doge.plot(style='', figsize=(15,5), color="#F8766D", title='DOGE Price (USD) by Days')

In [ ]:
#Plotting the test and training sets
_ = data_test.rename(columns={'Price': 'Test Set'}).join(data_train.rename(columns={'Price': 'Training Set'}), how='outer').plot(figsize=(15,5), title='Test & Training Sets', style='')

In [ ]:
#Creating the model
model = Sequential()
model.add(LSTM(128,activation="sigmoid",input_shape=(1,1)))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=100, batch_size=50, verbose=2)

In [ ]:
model.summary()

In [ ]:
#Making the predictions
test_set = data_test.values
inputs = np.reshape(test_set, (len(test_set), 1))
inputs = sc.transform(inputs)
inputs = np.reshape(inputs, (len(inputs), 1, 1))
predicted_DOGE_price = model.predict(inputs)
predicted_DOGE_price = sc.inverse_transform(predicted_DOGE_price)

In [ ]:
data_test['Price_Prediction'] = predicted_DOGE_price
data_all = pd.concat([data_test, data_train], sort=False)

In [ ]:
data_all

In [ ]:
_ = data_all[['Price','Price_Prediction']].plot(figsize=(15, 5))

In [ ]:
#Plotting the forecast v/s actual price
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
_ = data_all[['Price_Prediction','Price']].plot(ax=ax)
ax.set_xbound(lower='01-02-2021', upper='20-05-2021')
plot = plt.suptitle('Jan to May: Forecast vs Actual')

In [ ]:
#MSE
mean_squared_error(y_true=data_test['Price'],y_pred=data_test['Price_Prediction'])

In [ ]:
#MAE
mean_absolute_error(y_true=data_test['Price'],y_pred=data_test['Price_Prediction'])